In [18]:
# Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
pd.set_option('max_colwidth', 400)

In [19]:
# Importing Data Sets 
demographics_df = pd.read_csv('./Resources/NYC_Demographics.csv')
evictions_df = pd.read_csv('./Resources/NYC_Evictions.csv')

In [21]:
# Create a filtered dataframe from specific columns
demographics_cols = ["JURISDICTION NAME", "COUNT PARTICIPANTS", "PERCENT FEMALE", "PERCENT MALE", 
                     "PERCENT PACIFIC ISLANDER", "PERCENT HISPANIC LATINO", "PERCENT AMERICAN INDIAN", 
                     "PERCENT ASIAN NON HISPANIC", "PERCENT WHITE NON HISPANIC", "PERCENT BLACK NON HISPANIC", 
                     "PERCENT OTHER ETHNICITY", "PERCENT ETHNICITY UNKNOWN", "PERCENT PERMANENT RESIDENT ALIEN", 
                     "PERCENT US CITIZEN", "PERCENT OTHER CITIZEN STATUS", "PERCENT RECEIVES PUBLIC ASSISTANCE", 
                     "PERCENT NRECEIVES PUBLIC ASSISTANCE"]
demographics_filtered_df = demographics_df[demographics_cols]

# Rename the column headers
demographics_filtered_df = demographics_filtered_df.rename(columns={"JURISDICTION NAME": "ZIPCODE"})

# Clean the data by dropping duplicates and dropping zipcodes with no participants
demographics_filtered_df.drop_duplicates("ZIPCODE", inplace=True)
demographics_filtered = demographics_filtered_df.loc[demographics_filtered_df["COUNT PARTICIPANTS"] != 0]


demographics_filtered

,ZIPCODE,COUNT PARTICIPANTS,PERCENT FEMALE,PERCENT MALE,PERCENT PACIFIC ISLANDER,PERCENT HISPANIC LATINO,PERCENT AMERICAN INDIAN,PERCENT ASIAN NON HISPANIC,PERCENT WHITE NON HISPANIC,PERCENT BLACK NON HISPANIC,PERCENT OTHER ETHNICITY,PERCENT ETHNICITY UNKNOWN,PERCENT PERMANENT RESIDENT ALIEN,PERCENT US CITIZEN,PERCENT OTHER CITIZEN STATUS,PERCENT RECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE
0,10001,44,0.50,0.50,0.0,0.36,0.00,0.07,0.02,0.48,0.07,0.00,0.05,0.95,0.0,0.45,0.55
1,10002,35,0.54,0.46,0.0,0.03,0.00,0.80,0.17,0.00,0.00,0.00,0.06,0.94,0.0,0.06,0.94
2,10003,1,1.00,0.00,0.0,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.0,0.00,1.00
4,10005,2,1.00,0.00,0.0,0.00,0.00,0.50,0.00,0.50,0.00,0.00,0.50,0.50,0.0,0.00,1.00
5,10006,6,0.33,0.67,0.0,0.33,0.00,0.00,0.17,0.50,0.00,0.00,0.00,1.00,0.0,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,12783,201,0.33,0.67,0.0,0.00,0.01,0.00,0.97,0.00,0.01,0.00,0.02,0.98,0.0,0.38,0.62
230,12786,4,0.75,0.25,0.0,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.0,0.25,0.75
231,12788,83,0.47,0.53,0.0,0.00,0.00,0.00,0.98,0.00,0.02,0.00,0.00,1.00,0.0,0.42,0.58
232,12789,272,0.42,0.58,0.0,0.00,0.00,0.00,0.96,0.00,0.02,0.01,0.00,1.00,0.0,0.26,0.74


In [22]:
# Convert Executed Date to DateTime
evictions_df["Executed Date"]= pd.to_datetime(evictions_df["Executed Date"])

# Create a filtered dataframe from specific columns and dates in 
evictions_cols = ["Eviction Zip", "Executed Date", "Residential/Commercial"]
evictions_filtered_df = evictions_df[evictions_cols]

evictions_filtered = evictions_filtered_df.loc[evictions_filtered_df["Residential/Commercial"] == "Residential"]

evictions_filtered

,Eviction Zip,Executed Date,Residential/Commercial
0,10456,2018-02-26,Residential
1,10466,2022-11-16,Residential
2,10467,2017-09-29,Residential
3,11213,2018-07-12,Residential
4,10470,2019-10-24,Residential
...,...,...,...
72483,11368,2019-10-28,Residential
72484,11423,2019-08-06,Residential
72485,10472,2018-07-13,Residential
72486,11203,2019-12-04,Residential


In [13]:
# Creating Data Frame for evictions by zipcode 
zipcode_evictions = evictions_filtered.groupby(['Eviction Zip'])['Eviction Zip'].count()
zipcode_evictions_df = zipcode_evictions.to_frame()
zipcode_evictions_df = zipcode_evictions_df.rename(columns = {'Eviction Zip' : 'Total Evictions'})
zipcode_evictions_df.reset_index()

,Eviction Zip,Total Evictions
0,0,3
1,10000,1
2,10001,169
3,10002,267
4,10003,123
...,...,...
203,11692,369
204,11693,107
205,11694,143
206,11698,1


In [14]:
# Combine the data into a single dataset, left merging the demographics onto the evictions dataframe.  
combined_df = pd.merge(zipcode_evictions_df, demographics_filtered, left_on="Eviction Zip", right_on="ZIPCODE", how='inner')
combined_df#.count()

,Total Evictions,ZIPCODE,COUNT PARTICIPANTS,PERCENT FEMALE,PERCENT MALE,PERCENT PACIFIC ISLANDER,PERCENT HISPANIC LATINO,PERCENT AMERICAN INDIAN,PERCENT ASIAN NON HISPANIC,PERCENT WHITE NON HISPANIC,PERCENT BLACK NON HISPANIC,PERCENT OTHER ETHNICITY,PERCENT ETHNICITY UNKNOWN,PERCENT PERMANENT RESIDENT ALIEN,PERCENT US CITIZEN,PERCENT OTHER CITIZEN STATUS,PERCENT RECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE
0,169,10001,44,0.50,0.50,0.0,0.36,0.0,0.07,0.02,0.48,0.07,0.0,0.05,0.95,0.0,0.45,0.55
1,267,10002,35,0.54,0.46,0.0,0.03,0.0,0.80,0.17,0.00,0.00,0.0,0.06,0.94,0.0,0.06,0.94
2,123,10003,1,1.00,0.00,0.0,0.00,0.0,1.00,0.00,0.00,0.00,0.0,0.00,1.00,0.0,0.00,1.00
3,55,10005,2,1.00,0.00,0.0,0.00,0.0,0.50,0.00,0.50,0.00,0.0,0.50,0.50,0.0,0.00,1.00
4,23,10006,6,0.33,0.67,0.0,0.33,0.0,0.00,0.17,0.50,0.00,0.0,0.00,1.00,0.0,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,289,11420,2,0.50,0.50,0.0,0.50,0.0,0.00,0.00,0.00,0.50,0.0,0.00,1.00,0.0,1.00,0.00
76,109,11427,2,0.00,1.00,0.0,0.50,0.0,0.00,0.00,0.50,0.00,0.0,0.50,0.50,0.0,0.00,1.00
77,504,11432,2,1.00,0.00,0.0,0.00,0.0,0.50,0.00,0.50,0.00,0.0,0.00,1.00,0.0,0.00,1.00
78,618,11434,4,0.25,0.75,0.0,0.00,0.0,0.00,0.00,1.00,0.00,0.0,0.25,0.75,0.0,0.00,1.00
